# Frictionless SQLModel

<br>

### Imports

In [1]:
import requests

from powerdict import db

In [2]:
from IPython.display import JSON

<br>

### Frictionless Specifications

A Frictionless `DataPackage` contains one or more `DataResource`s that have a corresponding `DataSchema` with `FieldDescriptor`s that describe individual columns within datasets.

Here we'll load in the schema for a `DataPackage`

In [3]:
JSON(db.DataPackage.schema())

<IPython.core.display.JSON object>

<br>

We'll now load in an example frictionless dataset

In [4]:
def get_package_metadata(fd_url):
    r = requests.get(fd_url)
    r.raise_for_status()
    return r.json()

In [10]:
fd_url = 'https://pkgstore.datahub.io/core/gdp/9/datapackage.json'

raw_package_metadata = get_package_metadata(fd_url)
raw_package_metadata['version'] = f"0.1.{raw_package_metadata['version']}" # validation fix

JSON(raw_package_metadata)

<IPython.core.display.JSON object>

<br>

We can now parse and validate this raw package metadata using our schema

In [11]:
package_metadata = db.DataPackage.parse_obj(raw_package_metadata)

package_metadata

DataPackage(name='gdp', resources=[DataResource(name='validation_report', data=None, path=None, profile=<DataResourceProfile.data_resource: 'data-resource'>, title=None, description='Validation report for tabular data', format=<DataResourceFormat.json: 'json'>, mediatype=None, encoding='utf-8', bytes=591, hash='8500533ca81a0cd8993867c3bf11c30e', sources=None, licenses=None), DataResource(name='gdp_csv', data=None, path=None, profile=<DataResourceProfile.data_resource: 'data-resource'>, title=None, description=None, format=<DataResourceFormat.csv: 'csv'>, mediatype=None, encoding='utf-8', bytes=454342, hash='0048bc8f6228d0393d41cac4b663b90f', sources=None, licenses=None), DataResource(name='gdp_json', data=None, path=None, profile=<DataResourceProfile.data_resource: 'data-resource'>, title=None, description=None, format=<DataResourceFormat.json: 'json'>, mediatype=None, encoding='utf-8', bytes=1111120, hash='1a2503aa36368933be8f9a96e1dc16de', sources=None, licenses=None), DataResource(n

<br>

We'll do the same with another package

In [14]:
fd_url = 'https://pkgstore.datahub.io/core/s-and-p-500-companies/10/datapackage.json'

raw_package_metadata = get_package_metadata(fd_url)
raw_package_metadata['version'] = f"{raw_package_metadata['version']}.0" # validation fix

package_metadata = db.DataPackage.parse_obj(raw_package_metadata)

package_metadata

DataPackage(name='s-and-p-500-companies', resources=[DataResource(name='validation_report', data=None, path=None, profile=<DataResourceProfile.data_resource: 'data-resource'>, title=None, description='Validation report for tabular data', format=<DataResourceFormat.json: 'json'>, mediatype=None, encoding=None, bytes=448, hash='bb3cd3168933a568d27700f2099ad097', sources=None, licenses=None), DataResource(name='constituents_csv', data=None, path=None, profile=<DataResourceProfile.data_resource: 'data-resource'>, title=None, description=None, format=<DataResourceFormat.csv: 'csv'>, mediatype='text/csv', encoding='utf-8', bytes=19162, hash='b8d80d0c578007c9e0d199a6cd2625f1', sources=None, licenses=None), DataResource(name='constituents_json', data=None, path=None, profile=<DataResourceProfile.data_resource: 'data-resource'>, title=None, description=None, format=<DataResourceFormat.json: 'json'>, mediatype='text/csv', encoding='utf-8', bytes=37800, hash='64dd3e9582b936b0352fdd826ecd3c95', so